Merge Movie Financial Data with Genre Data in a new dataframe.  Missing data will be queried from TMDB using their API.  The resulting dataframe will be saved in a CSV file for analysis.

In [1]:
import pandas as pd
import requests
import json
import math
import warnings
import time

In [2]:
def get_keys(path):
    """Get the API Key in the file indictated by the path parameter"""
    with open(path) as f:
        return json.load(f)

# Read in the Data from its CSVs

In [3]:
last_20_years_movie_budgets_df = pd.read_csv('datasets/last_20_years_movie_budgets.csv')

In [4]:
clean_tmdb_movies_df = pd.read_csv('datasets/clean_tmdb_movies.csv')

# Merge Available Data

do a left join so that none of the budget/profit data is lost and then attempt to fill in the genre data

## Left Join

In [5]:
bud_prof_genres_df = pd.merge(last_20_years_movie_budgets_df, clean_tmdb_movies_df, how='left', 
                              left_on=['movie', 'release_year'], right_on=['original_title', 'release_year'])

## Look at the Data

In [6]:
bud_prof_genres_df[['movie','title', 
                    'real_production_budget', 'real_domestic_gross', 'real_worldwide_gross', 'real_profit',
                    'genre_ids']]

,movie,title,real_production_budget,real_domestic_gross,real_worldwide_gross,real_profit,genre_ids
0,My Dog Skip,NaN,1.039256e+07,5.062608e+07,5.314358e+07,4.275102e+07,NaN
1,Next Friday,NaN,1.410419e+07,8.488730e+07,8.859704e+07,7.449285e+07,NaN
2,The Terrorist,NaN,3.711629e+04,2.895709e+05,2.895709e+05,2.524546e+05,NaN
3,Supernova,NaN,8.907909e+07,2.111006e+07,2.199733e+07,-6.708176e+07,NaN
4,The Boondock Saints,NaN,1.039256e+07,4.523882e+04,6.114894e+05,-9.781072e+06,NaN
...,...,...,...,...,...,...,...
4000,Aladdin,NaN,1.820000e+08,2.467343e+08,6.192343e+08,4.372343e+08,NaN
4001,BrightBurn,NaN,7.000000e+06,1.679443e+07,2.798950e+07,2.098950e+07,NaN
4002,Rocketman,NaN,4.100000e+07,5.734272e+07,1.086427e+08,6.764272e+07,NaN
4003,Godzilla: King of the Monsters,NaN,1.700000e+08,8.557694e+07,2.992769e+08,1.292769e+08,NaN


Check to see how many rows are missing Genre Info

In [7]:
bud_prof_genres_df['genre_ids'].isna().sum()

2431

## Gather Missing Genre Ids

After doing a merge of the Budgets/Profit dataframe with the TMDB movies dataframe about half of the movies with a budget didn't merge with the TMDB dataframe

**Why?** Are they not in the TMDB dataframe, or is there a slight difference in the title?  Also, can the data be retrieved from the TMDB api to fill in the missing information?

In [8]:
# let's check the first movie
# are there any movies with the word 'dog' in the TMDB dataframe?
clean_tmdb_movies_df[clean_tmdb_movies_df['original_title'].str.contains('Skip|skip')]

,Unnamed: 0,Unnamed: 0.1,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,release_year
1294,1306,1306,[],473553,en,Sundance Skippy,1.114,2010-01-21,Sundance Skippy,8.0,5,2010
17416,17493,17493,"[28, 35, 53, 9648]",270774,en,Skiptrace,12.525,2016-09-02,Skiptrace,5.6,324,2016


In [9]:
clean_tmdb_movies_df[clean_tmdb_movies_df['title'].str.contains('Skip|skip')]

,Unnamed: 0,Unnamed: 0.1,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,release_year
1294,1306,1306,[],473553,en,Sundance Skippy,1.114,2010-01-21,Sundance Skippy,8.0,5,2010
17416,17493,17493,"[28, 35, 53, 9648]",270774,en,Skiptrace,12.525,2016-09-02,Skiptrace,5.6,324,2016


The first movie doesn't appeal to be in the TMDB dataframe, but it is on the TMDB site, so it can be queried to get its data

**What is the best way to get this data using the API?**

I will iterate through the new dataframe and query the API for the missing data, and then see what is still missing

In [10]:
bud_prof_genres_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4005 entries, 0 to 4004
Data columns (total 24 columns):
Unnamed: 0_x              4005 non-null int64
id_x                      4005 non-null int64
release_date_x            4005 non-null object
movie                     4005 non-null object
production_budget         4005 non-null int64
domestic_gross            4005 non-null int64
worldwide_gross           4005 non-null int64
release_year              4005 non-null int64
real_production_budget    4005 non-null float64
real_domestic_gross       4005 non-null float64
real_worldwide_gross      4005 non-null float64
real_profit               4005 non-null float64
real_margin               4005 non-null float64
Unnamed: 0_y              1574 non-null float64
Unnamed: 0.1              1574 non-null float64
genre_ids                 1574 non-null object
id_y                      1574 non-null float64
original_language         1574 non-null object
original_title            1574 non-null objec

### Create a mask for the missing Genre ID's

In [11]:
bud_prof_genres_df.genre_ids.unique()

array([nan, '[80, 18, 878, 53]', '[18, 10402]', '[28, 18, 53, 10752]',
       '[28, 18, 9648, 53]', '[80, 53]', '[27, 12, 28]', '[28, 18, 53]',
       '[27, 9648, 80]', '[878, 28, 27, 53, 12, 14]', '[12, 35, 10751]',
       '[18, 53]', '[10749, 35]', '[12, 14]', '[12, 16, 10751]', '[35]',
       '[28, 12, 14, 878]', '[28, 35, 10751]', '[28, 53, 878]',
       '[35, 10751, 14]', '[18]', '[27]', '[80, 18, 9648, 53]',
       '[18, 10749, 10752]', '[12, 14, 10751]', '[35, 10749]',
       '[53, 9648]', '[18, 53, 9648]', '[18, 28, 35, 80]', '[18, 80]',
       '[28, 35, 80]', '[9648, 27, 28]', '[10751, 14, 12]', '[]',
       '[10752, 28, 12, 18, 53]', '[18, 10749]', '[28, 35, 18]',
       '[28, 878, 53, 80]', '[35, 10751]', '[878, 35, 12]',
       '[80, 53, 35, 9648]', '[14, 12, 16, 10751]', '[18, 10751, 10749]',
       '[12, 14, 28]', '[35, 80, 53]', '[18, 10751]',
       '[80, 18, 9648, 10749]', '[35, 18]', '[28, 80]', '[99, 10751]',
       '[28, 12, 80, 9648, 53]', '[53, 80, 18, 28]', '[27,

In [12]:
nan = bud_prof_genres_df.genre_ids.unique()[0]

In [13]:
type(nan)

float

In [14]:
# check that nan will return True (it is NaN)
math.isnan(nan)

True

In [15]:
def is_nan(x):
    """Return Boolean of whether the input value is NaN"""
    if type(x) == type(1.9):
        return math.isnan(x)
    else:
        return False

In [16]:
mask = bud_prof_genres_df.genre_ids.apply(is_nan)

Look at the results of the mask

In [17]:
bud_prof_genres_df.genre_ids[mask].unique()

array([nan], dtype=object)

## Gather Data with API
Now that the mask has been created, it's time to gather the genre data using the TMDB API and add it to the active dataframe

In [18]:
# get the api key
keys = get_keys('/Users/freethrall/.secret/tmdb_api.json')
api_key = keys['api_key']

# url of the movie database api for genres
url = 'https://api.themoviedb.org/3/search/movie'

def get_genre_ids(movie):
    """Return the genre_ids of the parameter movie, will use TMDB API if neccessary"""
    
    movie_title = movie.movie

    parameters = {'api_key': api_key,
                 'language': 'en-US',
                 'query': movie_title}

    # get the current time
    t0 = time.time()
    
    r = requests.get(url, params=parameters)
    
    # the delay is the current time - t0 (the previous time)
    response_delay = time.time() - t0
    
    # wait 10x longer than it took them to respond
    time.sleep(10*response_delay)  
    
    if r.status_code != 200: #there was a problem
        warnings.warn(f'There was a problem with {movie_title}, status code {r.status_code}')
        return None
    else:
        results = pd.DataFrame.from_dict(r.json()['results'])
        
        if len(results) == 0: # the movie data is not available
            print(f'Genre Data not available for {movie_title}')
            return None
        else:
            genre_id_list = str(results['genre_ids'][0])
            #print(f'{movie_title} -- {genre_id_list}')      
            return genre_id_list
    
 

In [19]:
bud_prof_genres_df.genre_ids[mask] = bud_prof_genres_df[mask].apply(get_genre_ids, axis=1)
bud_prof_genres_df

Genre Data not available for Hum to Mohabbt Karega
Genre Data not available for AimÃ©e & Jaguar
Genre Data not available for The Widow of St. Pierre
Genre Data not available for A Knightâs Tale
Genre Data not available for John Carpenterâs Ghosts of Mars
Genre Data not available for Donât Say a Word
Genre Data not available for Le Fabuleux destin d'AmÃ©lie Poulain
Genre Data not available for Keeping it Real: The Adventures of Greg Walloch
Genre Data not available for Harry Potter and the Sorcererâs Stone
Genre Data not available for Y Tu MamÃ¡ TambiÃ©n
Genre Data not available for National Lampoonâs Van Wilder
Genre Data not available for The Business of Fancy Dancing
Genre Data not available for Star Wars Ep. II: Attack of the Clones
Genre Data not available for The Divine Secrets of the Ya-Ya Sisterhood
Genre Data not available for Wes Craven Presents: They
Genre Data not available for Steal (Canadian Release)
Genre Data not available for 28 Days Laterâ¦
Genre Data not av

/Users/freethrall/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0_x,id_x,release_date_x,movie,production_budget,domestic_gross,worldwide_gross,release_year,real_production_budget,real_domestic_gross,...,Unnamed: 0.1,genre_ids,id_y,original_language,original_title,popularity,release_date_y,title,vote_average,vote_count
0,3994,95,2000-01-12,My Dog Skip,7000000,34099640,35795319,2000,1.039256e+07,5.062608e+07,...,NaN,"[35, 18]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3752,53,2000-01-12,Next Friday,9500000,57176582,59675307,2000,1.410419e+07,8.488730e+07,...,NaN,[35],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5747,48,2000-01-14,The Terrorist,25000,195043,195043,2000,3.711629e+04,2.895709e+05,...,NaN,[99],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,940,41,2000-01-14,Supernova,60000000,14218868,14816494,2000,8.907909e+07,2.111006e+07,...,NaN,"[27, 878, 53]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4060,61,2000-01-21,The Boondock Saints,7000000,30471,411874,2000,1.039256e+07,4.523882e+04,...,NaN,"[28, 80, 53]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4000,80,81,2019-05-24,Aladdin,182000000,246734314,619234314,2019,1.820000e+08,2.467343e+08,...,NaN,"[12, 35, 14, 10749, 10751]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4001,4012,13,2019-05-24,BrightBurn,7000000,16794432,27989498,2019,7.000000e+06,1.679443e+07,...,NaN,"[27, 878, 53]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4002,1370,71,2019-05-31,Rocketman,41000000,57342725,108642725,2019,4.100000e+07,5.734272e+07,...,NaN,[18],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4003,124,25,2019-05-31,Godzilla: King of the Monsters,170000000,85576941,299276941,2019,1.700000e+08,8.557694e+07,...,NaN,"[28, 878]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


How many movies still don't have genre info?  Either as a NaN or []

Should the missing data still be collected or those rows dropped?

Collect data and drop leftover rows with missing data

Save the dataframe as a CSV file

In [20]:
bud_prof_genres_df.to_csv(r'datasets/financials_with_genres.csv')